In [2]:
import pandas as pd
data = pd.read_csv('ratings.csv')

In [3]:
data = data.dropna()

print(data.head())
print(data.info())

   userId  movieId  rating   timestamp
0       1      296     5.0  1147880044
1       1      306     3.5  1147868817
2       1      307     5.0  1147868828
3       1      665     5.0  1147878820
4       1      899     3.5  1147868510
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16521840 entries, 0 to 16521839
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 504.2 MB
None


In [ ]:
!pip install scikit-surprise
from surprise import Dataset
from surprise import Reader

# Assuming the columns in your dataset are 'user_id', 'movie_id', and 'rating'
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(data[['userId', 'movieId', 'rating']], reader)

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/packaging/markers.py", line 191, in _eval_op
    spec = Specifier("".join([op.serialize(), rhs]))
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/packaging/specifiers.py", line 98, in __init__
    raise InvalidSpecifier(f"Invalid specifier: '{spec}'")
pip._vendor.packaging.specifiers.InvalidSpecifier: Invalid specifier: '==dev-tensorflow'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/

In [ ]:

from surprise import KNNWithMeans
from surprise import accuracy
from surprise.model_selection import train_test_split

# Split the dataset into training and testing sets
trainset, testset = train_test_split(data, test_size=0.25)

# Train the model using the User-Based collaborative filtering method
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

# Make predictions on the test set
predictions = algo.test(testset)

In [ ]:
# Calculate the root mean squared error (RMSE) of the model's predictions
print(accuracy.rmse(predictions))

In [ ]:
def get_top_n_recommendations(user_id, n=10):
    # Get the user's ratings
    user_ratings = data.filter(user_id=user_id)

    # Get the movies the user has already rated
    watched_movies = user_ratings['movie_id'].tolist()

    # Get the predictions for the movies the user hasn't seen yet
    predictions = algo.test(user_ratings)

    # Sort the predictions by rating in descending order
    sorted_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)

    # Get the top-N movie recommendations
    top_n_recommendations = [pred.iid for pred in sorted_predictions if pred.iid not in watched_movies][:n]

    return top_n_recommendations

In [1]:
from flask import Flask, render_template, request

app = Flask(__name__)

# Placeholder for recommendation logic
def get_top_n_recommendations(user_id, n):
    # Your recommendation logic goes here
    # Replace this with your actual recommendation generation process
    return [f'Recommendation {i}' for i in range(1, n + 1)]

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/recommendations', methods=['POST'])
def recommendations():
    user_id = int(request.form['user_id'])
    n = int(request.form['n'])

    recommendations = get_top_n_recommendations(user_id, n)

    return render_template('recommendations.html', recommendations=recommendations)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
python app.py


